# Australia COVID‑19 Weekly (Cases & Deaths) — Jan 2021 to Dec 2023
This minimal notebook downloads **daily** data from Our World in Data, then aggregates to **weekly sums**
(weeks ending **Sunday**) so you get a **complete weekly grid** from **2021‑01‑03** to **2023‑12‑31**.
It outputs `Australia_COVID_Weekly_2021_2023.csv`.

In [6]:
import pandas as pd

# ---------- Config ----------
COUNTRY = "Australia"
START_DATE = pd.Timestamp("2021-01-01")
END_DATE = pd.Timestamp("2023-12-31")
WEEK_FREQ = "W-SUN"
OUT_CSV = "Australia_COVID_Weekly_2021_2023_with_Weekly_Vax.csv"

DAILY_URL = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"
VAX_URL = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv"

# ---------- Load Daily Cases & Deaths ----------
cols = ["location", "date", "new_cases", "new_deaths"]
daily = pd.read_csv(DAILY_URL, usecols=cols, parse_dates=["date"])
daily = daily.query("location == @COUNTRY and date <= @END_DATE").copy()

# Buffer to ensure first full week
buffer_start = START_DATE - pd.Timedelta(days=6)
daily = daily[daily["date"] >= buffer_start]

daily[["new_cases", "new_deaths"]] = daily[["new_cases","new_deaths"]].apply(pd.to_numeric, errors="coerce").fillna(0)

weekly = daily.set_index("date").resample(WEEK_FREQ).sum()[["new_cases", "new_deaths"]]

# Ensure full weekly coverage
first_week_end = pd.Timestamp("2021-01-03")
last_week_end = pd.Timestamp("2023-12-31")
full_weeks = pd.date_range(first_week_end, last_week_end, freq=WEEK_FREQ)
weekly = weekly.reindex(full_weeks, fill_value=0)
weekly.index.name = "week_end"

# ---------- Load Vaccination Data ----------
vax = pd.read_csv(VAX_URL, parse_dates=["date"])
vax = vax.query("location == @COUNTRY and date <= @END_DATE")[["date", "daily_vaccinations"]]

# Clean numeric
vax["daily_vaccinations"] = pd.to_numeric(vax["daily_vaccinations"], errors="coerce").fillna(0)

# Weekly vaccination sums
vax_weekly_counts = vax.set_index("date").resample(WEEK_FREQ).sum()[["daily_vaccinations"]]
vax_weekly_counts = vax_weekly_counts.reindex(full_weeks, fill_value=0)

# ---------- Combine into Final Weekly Table ----------
out = weekly.join(vax_weekly_counts)
out = out.rename(columns={
    "new_cases": "weekly_cases",
    "new_deaths": "weekly_deaths",
    "daily_vaccinations": "weekly_vaccinations"
})
out["week_start"] = out.index - pd.to_timedelta(6, unit="D")
out = out.reset_index()[["week_start", "week_end", "weekly_cases", "weekly_deaths", "weekly_vaccinations"]]
out[["weekly_cases", "weekly_deaths", "weekly_vaccinations"]] = out[["weekly_cases", "weekly_deaths", "weekly_vaccinations"]].astype(int)

# ---------- Save and Display ----------
out.to_csv(OUT_CSV, index=False)

pd.set_option("display.max_rows", None)
print(out)

print(f"\nSaved -> {OUT_CSV}")
print("Totals 2021–2023:")
print("  Cases:", out["weekly_cases"].sum())
print("  Deaths:", out["weekly_deaths"].sum())
print("  Vaccinations:", out["weekly_vaccinations"].sum())


    week_start   week_end  weekly_cases  weekly_deaths  weekly_vaccinations
0   2020-12-28 2021-01-03           187              2                    0
1   2021-01-04 2021-01-10           112              0                    0
2   2021-01-11 2021-01-17           113              1                    0
3   2021-01-18 2021-01-24            58              0                    0
4   2021-01-25 2021-01-31            45              0                    0
5   2021-02-01 2021-02-07            39              0                    0
6   2021-02-08 2021-02-14            48              0                    0
7   2021-02-15 2021-02-21            28              1                    0
8   2021-02-22 2021-02-28            44              0                33876
9   2021-03-01 2021-03-07            67              0                46305
10  2021-03-08 2021-03-14            80              1                68497
11  2021-03-15 2021-03-21            79              0                99344
12  2021-03-